# Fake news Detection

## Importing required libraries ###

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

### Inserting fake and real dataset

In [31]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [32]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [33]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Inserting a column called "class" for fake and real news dataset to categories fake and true news. 

In [34]:
df_fake["class"] = 0
df_true["class"] = 1

In [35]:
df_fake.shape, df_true.shape

((23481, 5), (22708, 5))

In [36]:
df = pd.concat([df_fake, df_true], axis =0 )
df.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [37]:
df.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

## checking for Null values

In [38]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
class      0
dtype: int64

#### Randomly shuffling the dataframe 

In [39]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [40]:
df.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [41]:
df.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [42]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [43]:
df["title"] = df["title"].apply(wordopt)

#### Defining dependent and independent variable as x and y

In [44]:
x = df["title"]
y = df["class"]

#### Splitting the dataset into training set and testing set. 

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

#### Convert text to vectors

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### XGBoost Classifier ###

In [48]:
import xgboost as xgb

In [49]:
XGB = xgb.XGBClassifier(random_state=42)

In [50]:
XGB_fit = XGB.fit(xv_train,y_train)

In [51]:
XGB_pred = XGB.predict(xv_test)

In [52]:
xgb_acc = accuracy_score(XGB_pred,y_test)
xgb_acc

0.9242424242424242

In [53]:
print(classification_report(y_test, XGB_pred))

              precision    recall  f1-score   support

           0       0.96      0.89      0.92      1171
           1       0.90      0.96      0.93      1139

    accuracy                           0.92      2310
   macro avg       0.93      0.92      0.92      2310
weighted avg       0.93      0.92      0.92      2310



## VOICE REGOGNITION 

In [54]:
import speech_recognition as sr
from pydub import AudioSegment
import moviepy.editor as mp

In [55]:
clip = mp.VideoFileClip("input_video.mp4")
clip.audio.write_audiofile("audio_input.mp3")

MoviePy - Writing audio in audio_input.mp3


MoviePy - Done.


In [56]:
UserVoiceRecognizer = sr.Recognizer()

try:
    mp3_file_path = r"audio_input.mp3"

    audio = AudioSegment.from_mp3(mp3_file_path)
    wav_file_path = 'output.wav'
    audio.export(wav_file_path, format='wav')

    with sr.AudioFile(wav_file_path) as source:
        UserVoiceRecognizer.adjust_for_ambient_noise(source, duration=0.5)
        UserVoiceInput = UserVoiceRecognizer.listen(source)

        UserVoiceInput_converted_to_Text = UserVoiceRecognizer.recognize_google(UserVoiceInput)
        UserInput = UserVoiceInput_converted_to_Text.lower()
        
        print(UserVoiceInput_converted_to_Text)

        output_text_file = 'output.txt'
        with open(output_text_file, 'w') as text_file:
            text_file.write(UserVoiceInput_converted_to_Text)

except KeyboardInterrupt:
    print('A Keyboard Interrupt encountered; Terminating the Program !!!')
except sr.UnknownValueError:
    print("No speech detected or could not be recognized.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Trump science tax government spending bills into law


# Model Testing With Manual Entry

### News

In [57]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "True News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_XGB = XGB.predict(new_xv_test)

    return print("\nXGB Prediction : {}".format(output_lable(pred_XGB[0])))

In [58]:
print(UserInput)
manual_testing(UserInput)

trump science tax government spending bills into law

XGB Prediction : True News


In [59]:
manual_testing("FBI Russia probe was helped by an Australian diplomat tip-off") 


XGB Prediction : True News
